In [4]:
"""
Multi-Agent Dynamic Grid World Environment
Created by: Ardianto Wibowo
"""

import numpy as np
import sys
import random

# Add the path to the 'env' folder to sys.path
sys.path.append('env')

from ma_gridworld import Env

class SearchAgent:
    def __init__(self, num_actions):
        self.num_actions = num_actions
        self.targets_seen = []
        self.memory = []  
        self.location_noise_probability = location_noise_probability
        
        
        


  


    def analyse_sensor_data(self, agent_id, coordinate_observation, sensor_data_observation):
        for i in range(len(sensor_data_observation)):
            for j in range(len(sensor_data_observation[i])):
                data = sensor_data_observation[i][j]
                location = [coordinate_observation[0] + j - len(sensor_data_observation[i])//2,
                           coordinate_observation[1] + i - len(sensor_data_observation)//2]
                if data != None and 'target_' + str(agent_id) in data:
                    if location not in self.targets_seen:
                        self.targets_seen.append(location)
                    # تحديث الذاكرة مع تمرير agent_id
                if data is not None and 'target_' in data:  # التحقق من وجود هدف
                    self.update_memory(location, data, agent_id)  # استدعاء update_memory
                    
                if is_noise_enabled and random.random() < location_noise_probability:
                    noisy_location = [location[0] + np.random.choice([-1, 0, 1]), location[1] + np.random.choice([-1, 0, 1])]
                    print(f"Agent {agent_id}: Noise applied. Original: {location}, Noisy: {noisy_location}")
                    location = noisy_location

                # إزالة الأهداف التي تم جمعها
                if location[0] == coordinate_observation[0] and location[1] == coordinate_observation[1]:
                    if location in self.targets_seen:
                        self.targets_seen.remove(location)








        
    def analyse_communication(self, comm_observation, agent_id):
        for comm in comm_observation:
            origin_location = comm[0]
            sensor_data_observation = comm[1]
            reported_by = comm[2]

            # طباعة البيانات المستلمة للتحقق
         

            for i in range(len(sensor_data_observation)):
                for j in range(len(sensor_data_observation[i])):
                    data = sensor_data_observation[i][j]
                    location = [origin_location[0] + j - len(sensor_data_observation[i])//2,
                               origin_location[1] + i - len(sensor_data_observation)//2]
                    if data != None and 'target_' + str(agent_id) in data:
                        if location not in self.targets_seen:
                            self.targets_seen.append(location)
                            print(f"Agent {agent_id}: Add target to {location} sent by Agent {reported_by}")                   

                           
                            
    def update_memory(self, location, data, agent_id):

        print(f"Agent {agent_id} is updating memory for location {location} with data {data}")
        if location not in [record["location"] for record in self.memory]:
            # add
            self.memory.append({"location": location, "data": data})
            print(f"Agent {agent_id} Updated Memory: {location} -> {data}")





    def select_action(self, coordinate_observation, agent_id):
        print(f"Agent {agent_id}: Current targets seen: {self.targets_seen}")
        if len(self.targets_seen) > 0:
            closest_target = None
            closest_target_distance = 999999
            for target_coordinate in self.targets_seen:
                horizontal_distance = target_coordinate[0] - coordinate_observation[0]
                vertical_distance = target_coordinate[1] - coordinate_observation[1]
                distance = abs(horizontal_distance) + abs(vertical_distance)
                if distance < closest_target_distance:
                    closest_target_distance = distance
                    closest_target = target_coordinate

            if closest_target:
                print(f"Agent {agent_id}: Moving towards target at {closest_target}")
                horizontal_distance = closest_target[0] - coordinate_observation[0]
                vertical_distance = closest_target[1] - coordinate_observation[1]
                if abs(horizontal_distance) >= abs(vertical_distance):
                    return 3 if horizontal_distance < 0 else 4
                else:
                    return 1 if vertical_distance < 0 else 2
        else:
            print(f"Agent {agent_id}: No targets seen, taking random action.")
            return np.random.choice(self.num_actions - 1) + 1





def get_action(agent_id, observation, num_actions, agents, env):
    """
    This method provides a random action chosen recognized by the ma-gridworld environment:
    1: up, 2: down, 3: left, 4: right, 0: stay
    """
    
    coordinate_observation = tuple(observation[0])  # Keep observation as (x, y) tuple

    # Optional observation data may be used, depend on the agent needs.
    win_state_observation = observation[1]
    sensor_data_observation = observation[2]
    comm_observation = observation[3]
    

    print(f"Observation for Agent {agent_id}: {observation}")

    # Check if communication data is available
    if comm_observation:
        agents[agent_id].analyse_communication(comm_observation, agent_id)
    else:
        print(f"Agent {agent_id}: No communication data available this step.")

    # Analyze sensor data
    agents[agent_id].analyse_sensor_data(agent_id, coordinate_observation, sensor_data_observation)
    
    # Select physical action
    physical_action = agents[agent_id].select_action(coordinate_observation, agent_id)

    # Define communication action
    if env.is_agent_silent:
        comm_action = []  # Communication action is set to be zero if agent silent
    else:
        comm_action = [coordinate_observation, sensor_data_observation, agent_id] 
    
    return (physical_action, comm_action)


def run(num_episodes, max_steps_per_episode, agents, num_actions, env):
    for episode in range(num_episodes):
        print(f"Starting episode {episode + 1}")
        observations = env.reset()  # Reset the environment at the start of each episode
        
        for agent in agents:
            agent.__init__(num_actions)
        
        done = [False] * env.num_agents  # Initialize 'done' as a list for each agent
        step_count = 0

        while not all(done) and step_count < max_steps_per_episode:  # Stop if all agents are done or max steps reached
            actions = []
            next_observations = []
            
            for agent_id in range(env.num_agents):
                observation = observations[agent_id]
                action = get_action(agent_id, observation, num_actions, agents, env)
                
                actions.append(action)
                next_observations.append(observation)

            next_observations, rewards, done = env.step(actions)  # Step in the environment

            observations = next_observations
            step_count += 1

            # Render the environment
            env.render()

            # طباعة النتائج على الشاشة
            print(f"Step {step_count}:")
            for agent_id in range(env.num_agents):
                print(
                    f"  Agent {agent_id}: Observation: {observations[agent_id]}, "
                    f"Action: {actions[agent_id]}, Reward: {rewards[agent_id]}, Done: {done[agent_id]}"
                )

        # طباعة عدد الخطوات عند نهاية الحلقة
        print(f"Episode {episode + 1} finished after {step_count} steps.\n")





if __name__ == "__main__":

    gsize=15 #grid size (square)
    gpixels=30 #grid cell size in pixels

    is_sensor_active = True #True:  Activate the sensory observation data
    sensory_size = 3 #'is_sensor_active' must be True. The value must be odd, if event will be converted to one level odd number above
    
    num_agents = 10 #the number of agents will be run in paralel
    num_obstacles = 0 #the number of obstacles
    is_single_target = False #True: all agents have a single target, False: each agent has their own target
    num_targets_per_agent = 5 #'is_single_target' must be true to have an effect
    
    is_agent_silent = False #True: communication among agents is allowed
    
    
    is_noise_enabled = True  # تفعيل الضوضاء
    location_noise_probability = 0.9

    num_episodes=1 #the number of episode will be run
    max_steps_per_episode=1000 #each episode will be stopped when max_step is reached

    eps_moving_targets = 10 #set this value greater than 'num_episodes' to keep the targets in a stationary position
    eps_moving_obstacles = 10 #set this value greater than 'num_episodes' to keep the obstacles in a stationary position

    render = True #True: render the animation into the screen (so far, it is still can not be deactivated)

    min_obstacle_distance_from_target = 1 #min grid distance of each obstacles relative to targets
    max_obstacle_distance_from_target = 5 #max grid distance of each obstacles relative to targets
    min_obstacle_distance_from_agents = 1 #min grid distance of each obstacles relative to agents

    reward_normal = -1 #reward value of normal steps
    reward_obstacle = -5 #reward value when hit an obstacle
    reward_target = 50 #reward value when reach the target

    is_totally_random = True #True: target and obstacles initial as well as movement position is always random on each call, False: only random at the beginning. 
    animation_speed = 0.1 #smaller is faster 
    is_destroy_environment = True #True: automatically close the animation after all episodes end.  

    # Initialize environment
    env = Env(
        num_agents=num_agents, num_targets_per_agent=num_targets_per_agent, num_obstacles=num_obstacles,
        eps_moving_obstacles=eps_moving_obstacles, eps_moving_targets=eps_moving_targets,
        is_agent_silent=is_agent_silent, is_single_target=is_single_target, sensory_size=sensory_size,
        gpixels=gpixels, gheight=gsize, gwidth=gsize, is_sensor_active=is_sensor_active,
        min_obstacle_distance_from_target=min_obstacle_distance_from_target,
        max_obstacle_distance_from_target=max_obstacle_distance_from_target,
        min_obstacle_distance_from_agents=min_obstacle_distance_from_agents,
        is_totally_random=is_totally_random, animation_speed=animation_speed,
        reward_normal=reward_normal, reward_obstacle=reward_obstacle, reward_target=reward_target
    )
    
    num_actions = len(env.action_space)
    
    # Initialize Q-learning agents
    agents = [SearchAgent(num_actions) for _ in range(num_agents)]

    # Run episodes
    run(num_episodes, max_steps_per_episode, agents, num_actions, env)

    if is_destroy_environment:
        env.destroy_environment() 


Starting episode 1
Observation for Agent 0: [[0, 0], False, [[None, None, None], [None, 'agent', 'empty'], [None, 'empty', 'empty']], []]
Agent 0: No communication data available this step.
Agent 0: Noise applied. Original: [-1, -1], Noisy: [0, -1]
Agent 0: Noise applied. Original: [0, -1], Noisy: [-1, -1]
Agent 0: Noise applied. Original: [1, -1], Noisy: [2, -2]
Agent 0: Noise applied. Original: [-1, 0], Noisy: [-1, 1]
Agent 0: Noise applied. Original: [0, 0], Noisy: [-1, 0]
Agent 0: Noise applied. Original: [1, 0], Noisy: [0, -1]
Agent 0: Noise applied. Original: [-1, 1], Noisy: [0, 0]
Agent 0: Noise applied. Original: [0, 1], Noisy: [1, 2]
Agent 0: Noise applied. Original: [1, 1], Noisy: [2, 0]
Agent 0: Current targets seen: []
Agent 0: No targets seen, taking random action.
Observation for Agent 1: [[14, 0], False, [[None, None, None], ['empty', 'agent', None], ['empty', 'empty', None]], []]
Agent 1: No communication data available this step.
Agent 1: Noise applied. Original: [13, 

Step 2:
  Agent 0: Observation: [[1, 0], False, [[None, None, None], ['agent', 'empty', 'empty'], ['empty', 'empty', 'empty']], [[(14, 1), [['empty', 'agent', None], ['empty', 'empty', None], ['target_4_0', 'empty', None]], 1], [(14, 13), [['empty', 'empty', None], ['empty', 'empty', None], ['empty', 'agent', None]], 2], [(0, 14), [[None, 'empty', 'empty'], [None, 'agent', 'empty'], [None, None, None]], 3], [(6, 0), [[None, None, None], ['agent', 'empty', 'empty'], ['empty', 'empty', 'empty']], 4], [(8, 0), [[None, None, None], ['empty', 'empty', 'agent'], ['empty', 'target_7_4', 'target_7_1']], 5], [(14, 4), [['target_8_2', 'empty', None], ['target_2_4', 'empty', None], ['target_5_2', 'agent', None]], 6], [(14, 10), [['empty', 'agent', None], ['empty', 'empty', None], ['empty', 'empty', None]], 7], [(8, 14), [['empty', 'empty', 'target_9_0'], ['agent', 'empty', 'empty'], [None, None, None]], 8], [(0, 6), [[None, 'empty', 'empty'], [None, 'empty', 'empty'], [None, 'agent', 'empty']], 9

Step 4:
  Agent 0: Observation: [[0, 1], False, [[None, 'empty', 'empty'], [None, 'empty', 'agent'], [None, 'empty', 'empty']], [[(14, 2), [['empty', 'agent', None], ['target_4_0', 'empty', None], ['target_8_2', 'empty', None]], 1], [(14, 11), [['empty', 'empty', None], ['empty', 'empty', None], ['empty', 'agent', None]], 2], [(2, 14), [['empty', 'target_5_4', 'empty'], ['agent', 'empty', 'empty'], [None, None, None]], 3], [(7, 0), [[None, None, None], ['agent', 'empty', 'empty'], ['empty', 'empty', 'target_7_4']], 4], [(9, 1), [['empty', 'agent', 'empty'], ['target_7_4', 'target_7_1', 'empty'], ['empty', 'empty', 'empty']], 5], [(13, 5), [['empty', 'agent', 'empty'], ['empty', 'target_5_2', 'empty'], ['empty', 'empty', 'empty']], 6], [(14, 8), [['target_8_3', 'empty', None], ['empty', 'empty', None], ['empty', 'agent', None]], 7], [(9, 13), [['empty', 'empty', 'target_6_0'], ['empty', 'target_9_0', 'empty'], ['empty', 'agent', 'empty']], 8], [(1, 6), [['empty', 'empty', 'empty'], ['ag

Step 6:
  Agent 0: Observation: [[0, 3], False, [[None, 'agent', 'empty'], [None, 'empty', 'empty'], [None, 'empty', 'empty']], [[(12, 2), [['empty', 'target_1_0', 'empty'], ['target_3_0', 'target_9_3', 'agent'], ['empty', 'empty', 'target_8_2']], 1], [(14, 9), [['empty', 'empty', None], ['empty', 'empty', None], ['empty', 'agent', None]], 2], [(4, 14), [['empty', 'target_5_1', 'empty'], ['agent', 'empty', 'empty'], [None, None, None]], 3], [(9, 0), [[None, None, None], ['agent', 'empty', 'empty'], ['target_7_4', 'target_7_1', 'agent']], 4], [(10, 2), [['target_7_1', 'agent', 'empty'], ['empty', 'empty', 'target_3_0'], ['empty', 'target_1_1', 'empty']], 5], [(13, 7), [['empty', 'agent', 'empty'], ['empty', 'target_8_3', 'agent'], ['empty', 'empty', 'empty']], 6], [(14, 6), [['target_5_2', 'empty', None], ['empty', 'empty', None], ['agent', 'agent', None]], 7], [(9, 11), [['empty', 'target_4_2', 'empty'], ['empty', 'empty', 'empty'], ['empty', 'agent', 'target_6_0']], 8], [(3, 6), [['em

Step 8:
  Agent 0: Observation: [[2, 3], False, [['empty', 'empty', 'empty'], ['agent', 'empty', 'target_3_4'], ['empty', 'target_5_0', 'empty']], [[(11, 1), [['agent', 'empty', 'empty'], ['empty', 'empty', 'agent'], ['empty', 'agent', 'target_9_3']], 1], [(14, 7), [['agent', 'empty', None], ['target_8_3', 'empty', None], ['agent', 'agent', None]], 2], [(6, 14), [['empty', 'target_3_2', 'empty'], ['agent', 'empty', 'empty'], [None, None, None]], 3], [(11, 0), [[None, None, None], ['agent', 'empty', 'empty'], ['empty', 'agent', 'empty']], 4], [(11, 3), [['empty', 'agent', 'target_9_3'], ['target_1_0', 'empty', 'empty'], ['empty', 'empty', 'empty']], 5], [(13, 9), [['empty', 'agent', 'empty'], ['empty', 'empty', 'empty'], ['target_4_4', 'empty', 'empty']], 6], [(13, 5), [['empty', 'target_2_4', 'empty'], ['empty', 'target_5_2', 'empty'], ['empty', 'agent', 'empty']], 7], [(11, 11), [['empty', 'empty', 'target_4_4'], ['agent', 'empty', 'empty'], ['target_6_0', 'empty', 'target_2_0']], 8],

...

Step 56:
  Agent 0: Observation: [[6, 7], False, [['empty', 'agent', 'empty'], ['empty', 'empty', 'empty'], ['empty', 'empty', 'empty']], [[(3, 8), [['empty', 'empty', 'empty'], ['empty', 'empty', 'empty'], ['empty', 'agent', 'agent']], 1], [(5, 13), [['empty', 'agent', 'empty'], ['empty', 'empty', 'agent'], ['empty', 'empty', 'empty']], 2], [(3, 9), [['empty', 'agent', 'empty'], ['empty', 'empty', 'agent'], ['empty', 'empty', 'empty']], 3], [(1, 10), [['empty', 'agent', 'empty'], ['empty', 'target_2_0', 'empty'], ['empty', 'empty', 'empty']], 4], [(1, 3), [['empty', 'empty', 'empty'], ['empty', 'empty', 'empty'], ['empty', 'agent', 'empty']], 5], [(8, 10), [['empty', 'empty', 'empty'], ['empty', 'empty', 'empty'], ['empty', 'agent', 'empty']], 6], [(5, 13), [['empty', 'agent', 'empty'], ['empty', 'agent', 'empty'], ['empty', 'empty', 'empty']], 7], [(9, 2), [['empty', 'empty', 'empty'], ['empty', 'empty', 'agent'], ['empty', 'empty', 'empty']], 8], [(13, 2), [['empty', 'empty', 'empty

Step 58:
  Agent 0: Observation: [[5, 6], False, [['empty', 'empty', 'empty'], ['empty', 'empty', 'agent'], ['empty', 'empty', 'empty']], [[(2, 7), [['empty', 'empty', 'empty'], ['empty', 'empty', 'empty'], ['empty', 'agent', 'empty']], 1], [(3, 13), [['empty', 'empty', 'empty'], ['empty', 'empty', 'agent'], ['empty', 'empty', 'empty']], 2], [(2, 8), [['empty', 'agent', 'empty'], ['empty', 'empty', 'empty'], ['empty', 'agent', 'empty']], 3], [(2, 9), [['empty', 'agent', 'empty'], ['empty', 'empty', 'empty'], ['target_2_0', 'agent', 'empty']], 4], [(3, 3), [['empty', 'empty', 'empty'], ['agent', 'empty', 'empty'], ['empty', 'empty', 'empty']], 5], [(8, 12), [['empty', 'agent', 'empty'], ['empty', 'empty', 'empty'], ['empty', 'empty', 'empty']], 6], [(4, 14), [['agent', 'empty', 'empty'], ['empty', 'empty', 'agent'], [None, None, None]], 7], [(10, 1), [['empty', 'empty', 'empty'], ['empty', 'empty', 'empty'], ['empty', 'agent', 'empty']], 8], [(13, 2), [['empty', 'empty', 'empty'], ['emp

Step 60:
  Agent 0: Observation: [[5, 4], False, [['empty', 'empty', 'empty'], ['agent', 'empty', 'empty'], ['empty', 'agent', 'empty']], [[(3, 8), [['empty', 'empty', 'empty'], ['agent', 'empty', 'empty'], ['empty', 'empty', 'empty']], 1], [(2, 12), [['empty', 'empty', 'empty'], ['empty', 'empty', 'agent'], ['empty', 'empty', 'empty']], 2], [(0, 8), [[None, 'empty', 'empty'], [None, 'empty', 'agent'], [None, 'empty', 'empty']], 3], [(1, 8), [['empty', 'empty', 'empty'], ['agent', 'empty', 'agent'], ['empty', 'empty', 'empty']], 4], [(4, 4), [['empty', 'empty', 'empty'], ['agent', 'empty', 'empty'], ['empty', 'empty', 'agent']], 5], [(9, 13), [['empty', 'empty', 'empty'], ['agent', 'empty', 'empty'], ['empty', 'empty', 'empty']], 6], [(5, 13), [['empty', 'empty', 'empty'], ['empty', 'empty', 'empty'], ['empty', 'agent', 'empty']], 7], [(9, 0), [[None, None, None], ['empty', 'empty', 'agent'], ['empty', 'empty', 'empty']], 8], [(12, 1), [['empty', 'empty', 'empty'], ['empty', 'empty', '

Step 62:
  Agent 0: Observation: [[4, 3], False, [['empty', 'empty', 'empty'], ['empty', 'empty', 'agent'], ['empty', 'empty', 'empty']], [[(3, 8), [['empty', 'agent', 'empty'], ['empty', 'empty', 'empty'], ['empty', 'empty', 'empty']], 1], [(1, 11), [['empty', 'target_2_0', 'empty'], ['empty', 'empty', 'agent'], ['empty', 'empty', 'empty']], 2], [(0, 8), [[None, 'empty', 'agent'], [None, 'empty', 'agent'], [None, 'empty', 'empty']], 3], [(1, 8), [['empty', 'agent', 'empty'], ['agent', 'empty', 'empty'], ['empty', 'empty', 'empty']], 4], [(5, 5), [['empty', 'empty', 'empty'], ['agent', 'empty', 'empty'], ['empty', 'empty', 'empty']], 5], [(8, 12), [['empty', 'empty', 'empty'], ['empty', 'empty', 'agent'], ['empty', 'empty', 'empty']], 6], [(6, 14), [['empty', 'empty', 'empty'], ['agent', 'empty', 'empty'], [None, None, None]], 7], [(10, 1), [['empty', 'empty', 'empty'], ['agent', 'empty', 'agent'], ['empty', 'empty', 'empty']], 8], [(11, 0), [[None, None, None], ['empty', 'empty', 'emp

Step 64:
  Agent 0: Observation: [[3, 3], True, [['empty', 'empty', 'empty'], ['empty', 'agent', 'empty'], ['empty', 'empty', 'empty']], [[(3, 7), [['empty', 'empty', 'empty'], ['empty', 'empty', 'empty'], ['empty', 'agent', 'empty']], 1], [(1, 10), [['empty', 'empty', 'empty'], ['empty', 'empty', 'empty'], ['empty', 'agent', 'empty']], 2], [(0, 9), [[None, 'agent', 'agent'], [None, 'empty', 'empty'], [None, 'empty', 'agent']], 3], [(1, 9), [['empty', 'agent', 'empty'], ['agent', 'empty', 'empty'], ['empty', 'agent', 'empty']], 4], [(5, 4), [['agent', 'empty', 'empty'], ['empty', 'empty', 'empty'], ['empty', 'agent', 'empty']], 5], [(7, 12), [['empty', 'empty', 'empty'], ['empty', 'empty', 'agent'], ['empty', 'empty', 'empty']], 6], [(7, 14), [['empty', 'empty', 'empty'], ['agent', 'empty', 'empty'], [None, None, None]], 7], [(11, 1), [['empty', 'agent', 'empty'], ['agent', 'empty', 'empty'], ['empty', 'empty', 'empty']], 8], [(11, 0), [[None, None, None], ['empty', 'agent', 'empty'], 